In [1]:
from bs4 import BeautifulSoup
import requests

In [202]:
base_url ='http://www.elcats.ru/'

In [203]:
page = requests.get('http://www.elcats.ru/')

In [204]:
soup = BeautifulSoup(page.text, "html.parser")

In [209]:
car_hrefs = soup.findAll('a', class_ = 'menu1')[::-1]

cars = {a.text: 'http://www.elcats.ru' + a['href'] for a in car_hrefs 
        if 'www' not in a.text and 
                'http' not in a['href']
       }
cars

{'SCANIA': 'http://www.elcats.ru/truck/scania/',
 'RENAULT': 'http://www.elcats.ru/renault/',
 'MERCEDES': 'http://www.elcats.ru/mercedes/',
 'FIAT': 'http://www.elcats.ru/fiat/',
 'ZAZ CHANCE': 'http://www.elcats.ru/zaz/',
 'VOLVO': 'http://www.elcats.ru/volvo/',
 'VOLKSWAGEN': 'http://www.elcats.ru/vw/',
 'SMART': 'http://www.elcats.ru/smart/',
 'SKODA': 'http://www.elcats.ru/skoda/',
 'SEAT': 'http://www.elcats.ru/seat/',
 'SATURN': 'http://www.elcats.ru/gm/saturn/',
 'SAAB': 'http://www.elcats.ru/saab/',
 'PORSCHE': 'http://www.elcats.ru/porsche/',
 'PONTIAC': 'http://www.elcats.ru/gm/pontiac/',
 'PEUGEOT': 'http://www.elcats.ru/peugeot/',
 'OPEL': 'http://www.elcats.ru/opel/',
 'MINI': 'http://www.elcats.ru/mini/',
 'MG ROVER': 'http://www.elcats.ru/mgrover/',
 'LIFAN': 'http://www.elcats.ru/lifan/',
 'LAND ROVER': 'http://www.elcats.ru/landrover/',
 'LANCIA': 'http://www.elcats.ru/lancia/',
 'KIA': 'http://www.elcats.ru/kia/',
 'JEEP': 'http://www.elcats.ru/jeep/',
 'JAGUAR': 'ht

In [140]:
page = requests.get(cars['ABARTH'])
soup1 = BeautifulSoup(page.text, "html.parser")

items = soup1.findAll('a', href=lambda x: x and 'ToggleNode' in x, text=lambda x: x)
sub_lists = soup1.findAll('div', id=lambda x: x and x[0]=='d' and len(x) < 5)

In [143]:
len(sub_lists)

2

### Создание url из href кнопок

In [158]:
js_command = sub_lists[0].find('a')['href']
js_command

"javascript:submit('1002','1161','1002','0');"

In [166]:
import re
raw_submit_args = re.findall('javascript:(submit|subm)\((.*)\)', js_command)[0][1]

In [167]:
raw_submit_args

"'1002','1161','1002','0'"

In [115]:
import ast
pseudo_python_list = f'[{raw_submit_args}]'
submit_args = ast.literal_eval(pseudo_python_list)
submit_args

['1002', '1161', '1002', '0']

In [42]:
sumbit_function_body = soup1.find(text=lambda t: 'function submit' in t)
print(sumbit_function_body)


        function submit(modelId, yearId, region, value)
        {
            var url = document.location.href;
            if (document.location.href.indexOf('#') != -1)
                url = document.location.href.substring(0, document.location.href.indexOf('#'));
            if (value != '')
                document.location.href = url + "#" + value;
            else
                document.location.href = url;
            var form = document.getElementById('PostForm');
            form.innerHTML = "<input type=\"hidden\" name=\"Model\">";
            form.innerHTML += "<input type=\"hidden\" name=\"Year\">";
            form.innerHTML += "<input type=\"hidden\" name=\"Region\">";
            document.forms[1].Model.value = modelId;
            document.forms[1].Year.value = yearId;
            document.forms[1].Region.value = region;
            document.forms[1].action = "Modification.aspx";
            document.forms[1].submit();
        }
    


In [116]:
submit_params = re.findall('\[1\]\.(.*)\.value =.*;', sumbit_function_body)
submit_params

['Model', 'Year', 'Region']

In [117]:
submit_action = re.findall('\.action = "(.*)"', sumbit_function_body)[0]
submit_action

'Modification.aspx'

In [118]:
dict(zip(submit_params, submit_args))

{'Model': '1002', 'Year': '1161', 'Region': '1002'}

In [119]:
from urllib.parse import urlencode
from collections import OrderedDict

url_params_line = urlencode(OrderedDict(**dict(zip(submit_params, submit_args))))

f'{base_url}volvo/{submit_action}?{url_params_line}'

'http://www.elcats.ru/volvo/Modification.aspx?Model=1002&Year=1161&Region=1002'

In [176]:
def create_url_from_href(page_soup, base_url, command):
    raw_submit_args = re.findall('javascript:(submit|subm)\((.*)\)', command)
    # если находим шаблон
    if raw_submit_args:
        raw_submit_args = raw_submit_args[0][1]
        pseudo_python_list = f'[{raw_submit_args}]'
        submit_args = ast.literal_eval(pseudo_python_list)
        sumbit_function_body = page_soup.find(text=lambda t: 'function submit' in t)
        submit_params = re.findall('\[1\]\.(.*)\.value =.*;', sumbit_function_body)
        submit_action = re.findall('\.action = "(.*)"', sumbit_function_body)[0]
        url_params_line = urlencode(OrderedDict(**dict(zip(submit_params, submit_args))))
        return f'{base_url}{submit_action}?{url_params_line}'

### Парсинг раскрывающихся списков

In [130]:
[(a.text, create_url_from_href(soup1, 'volvo', a['href'])) for a in  sub_lists[0].findAll('a')]

[('1975',
  'http://www.elcats.ru/volvo/Modification.aspx?Model=1002&Year=1161&Region=1002'),
 ('1976',
  'http://www.elcats.ru/volvo/Modification.aspx?Model=1002&Year=1162&Region=1002'),
 ('1977',
  'http://www.elcats.ru/volvo/Modification.aspx?Model=1002&Year=1163&Region=1002'),
 ('1978',
  'http://www.elcats.ru/volvo/Modification.aspx?Model=1002&Year=1164&Region=1002'),
 ('1979',
  'http://www.elcats.ru/volvo/Modification.aspx?Model=1002&Year=1165&Region=1002'),
 ('1980',
  'http://www.elcats.ru/volvo/Modification.aspx?Model=1002&Year=1166&Region=1002'),
 ('1981',
  'http://www.elcats.ru/volvo/Modification.aspx?Model=1002&Year=1167&Region=1002'),
 ('1982',
  'http://www.elcats.ru/volvo/Modification.aspx?Model=1002&Year=1168&Region=1002'),
 ('1983',
  'http://www.elcats.ru/volvo/Modification.aspx?Model=1002&Year=1169&Region=1002'),
 ('1984',
  'http://www.elcats.ru/volvo/Modification.aspx?Model=1002&Year=1170&Region=1002'),
 ('1985',
  'http://www.elcats.ru/volvo/Modification.aspx?Mo

In [ ]:
{
    i.text: {a.text: create_url_from_href(soup1, 'volvo', a['href']) for a in sl.findAll('a')}
    for i, sl in zip(items, sub_lists)
}

### Парсинг обычных списков

In [ ]:
page = requests.get(cars['BMW'])
soup1 = BeautifulSoup(page.text, "html.parser")

flat_items = soup1.findAll('a', href=lambda x: x and 'javascript' in x, text=lambda x: x)
{a.text: create_url_from_href(soup1, 'BMW', a['href']) for a in flat_items}

### Первый уровень: модели авто

In [177]:
def parse_page(soup_object, base_url, car_brand):
    items = soup_object.findAll('a', href=lambda x: x and 'ToggleNode' in x, text=lambda x: x)
    if items:
        sub_lists = soup.findAll('div', id=lambda x: x and x[0]=='d' and len(x) < 5)
        return {
            i.text: {a.text: create_url_from_href(soup_object, base_url, a['href']) for a in sl.findAll('a')}
            for i, sl in zip(items, sub_lists)
        }
    else:
        flat_items = soup_object.findAll('a', href=lambda x: x and 'javascript' in x, text=lambda x: x)
        return {a.text: create_url_from_href(soup_object, base_url, a['href']) for a in flat_items}

In [210]:
cars_extended = {}
for brand in cars: 
    base_url = cars[brand]
    page = requests.get(cars[brand].lower())
    soup = BeautifulSoup(page.text, "html.parser")
    cars_extended[brand] = parse_page(soup, base_url, brand)

In [196]:
def dict_flatten(my_dict):
    new_dict = {}
    for x in my_dict:
        if type(my_dict[x]) == dict:
            for y in my_dict[x]:
                new_dict[f'{x} >>> {y}'] = my_dict[x][y]
        else:
            new_dict[x] = my_dict[x]
    return new_dict

In [225]:
import random
composite_keys = list(dict_flatten(dict_flatten(cars_extended)).keys())
random.shuffle(composite_keys)
composite_keys[:5]

['AUDI >>> Audi RS4/Avant quattro (2000-2023) >>> 2016\xa0\xa0\xa0',
 'FIAT >>> DOBLÒ >>> NUOVO DOBLO 2009 (2009-....)',
 'RENAULT >>> Master II >>> UD1G',
 'BMW >>> 2.5CS MКПП',
 'RENAULT >>> Master I >>> QH94']

In [226]:
cars_extended['AUDI']['Audi RS4/Avant quattro (2000-2023)']

{'2000\xa0\xa0\xa0': 'http://www.elcats.ru/audi/Groups.aspx?Mdl=df57cffb-7855-4fa1-98b0-80d7e2877b1b',
 '2001\xa0\xa0\xa0': 'http://www.elcats.ru/audi/Groups.aspx?Mdl=2afbc3a5-ca8b-4e18-b7d8-0607a25dac82',
 '2002\xa0\xa0\xa0': 'http://www.elcats.ru/audi/Groups.aspx?Mdl=782ebc87-2436-43f1-b1d0-2c2745327950',
 '2006\xa0\xa0\xa0': 'http://www.elcats.ru/audi/Groups.aspx?Mdl=0143c108-1c92-402e-a162-3d9fbb5ececa',
 '2007\xa0\xa0\xa0': 'http://www.elcats.ru/audi/Groups.aspx?Mdl=41fc1f84-ba02-4125-9aaa-bc916ea7047c',
 '2008\xa0\xa0\xa0': 'http://www.elcats.ru/audi/Groups.aspx?Mdl=e252580e-b87f-423e-86a7-27e483381cf4',
 '2013\xa0\xa0\xa0': 'http://www.elcats.ru/audi/Groups.aspx?Mdl=f419982b-ed5e-4a7c-b9f6-67672b38745f',
 '2014\xa0\xa0\xa0': 'http://www.elcats.ru/audi/Groups.aspx?Mdl=5c08f572-43fa-4fc5-aafe-d6cfcbb84570',
 '2015\xa0\xa0\xa0': 'http://www.elcats.ru/audi/Groups.aspx?Mdl=a18ffeb8-c3fc-490f-8c1a-3f3f7c0e1eae',
 '2016\xa0\xa0\xa0': 'http://www.elcats.ru/audi/Groups.aspx?Mdl=c4628f34-

In [227]:
cars_extended['FIAT']['DOBLÒ']

{'NUOVO DOBLO 2009 (2009-....)': 'http://www.elcats.ru/fiat/Groups.aspx?Key=fURwSCwXttrdHjyFlSPTkA%3D%3D&Title=28',
 'DOBLO REST. 2005 (2005-....)': 'http://www.elcats.ru/fiat/Groups.aspx?Key=bEMsmJCG%2FVbBTXvKpSC2vg%3D%3D&Title=28',
 'DOBLO MY. 2004 (2004-2005)': 'http://www.elcats.ru/fiat/Groups.aspx?Key=vlyvpq45n17%2BidO%2FIg35dA%3D%3D&Title=28',
 'DOBLO (2000-2004)': 'http://www.elcats.ru/fiat/Groups.aspx?Key=QhHWK%2F1cl8USkB1t5alolQ%3D%3D&Title=28'}

In [229]:
cars_extended['BMW']['2.5CS MКПП']

'http://www.elcats.ru/bmw/MdlYear.aspx?Mdl=03400023&Kpp=M&T=25'